In [2]:
%load_ext autoreload

%autoreload 3

In [3]:
%reload_ext autoreload

In [9]:
!pip install wget
!python -m wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_win32.zip

You should consider upgrading via the 'C:\Users\user\.virtualenvs\google-arts-0o2XoGZ3\Scripts\python.exe -m pip install --upgrade pip' command.



Saved under chromedriver_win32.zip


# Packages

In [5]:
import pyprojroot
from pathlib  import Path
import time
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
from src.scraping.scraping_pictures import ScrapingPictures
from src.scraping.scraping_picture_info import ScrapingPictureInfo
from src.scraping.get_img import getImage


# Paths

In [7]:
root_path =  pyprojroot.here()
path_driver = (root_path / "notebooks"/'chromedriver.exe').relative_to('/')
data_raw_folder = (root_path / "data"/'raw')
data_processed_folder = (root_path / "data"/'processed')

# Scraping

In [23]:
page_sections = ["WHITE","PINK","YELLOW","PURPLE","BLUE","TEAL","GREEN","ORANGE","RED","BROWN","BLACK"] #Color images to be scraped
scroll_down_times = 120 #Num of scroll for charge more images

In [24]:
def scrapePictures(color) -> pd.DataFrame:
    """
    Scrapes the pictures of the given color from the website.
    """
    scraper = ScrapingPictures(path_driver,color)
    scraper.open()
    scraper.scroll_down(scroll_down_times)
    time.sleep(8)
    df = scraper.scrape()
    return df

In [19]:
def parallelScrapePictures() -> pd.DataFrame:
    """
    Scrape all the pictures from the website using multiprocessing
    """
    with ProcessPoolExecutor() as executor:
        scrape_df_list = executor.map(scrapePictures, page_sections)
        scrape_df = pd.concat(list(scrape_df_list), ignore_index=True)
        return scrape_df

In [26]:
t1 = time.perf_counter()
df = parallelScrapePictures() 
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Open page section YELLOW
Open page section BLUE
Open page section PINK
Open page section TEAL
Open page section BLACK
Open page section GREEN
Open page section RED
Open page section BROWN
Open page section ORANGE
Open page section PURPLE
Open page section WHITE
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling downFinish scrolling down

Finished in 373.1641500099995 seconds


In [27]:
len(df)

16378

In [30]:
df.sample(15)

,image,page,color
9526,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/escrav...,BROWN
6055,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/landsc...,GREEN
6703,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/fenest...,GREEN
10053,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/pierro...,BROWN
3718,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/hostag...,BLUE
12819,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/a-stor...,BLACK
2353,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/white-...,PURPLE
1649,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/powley...,YELLOW
15130,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/self-p...,BLACK
16313,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/gassed...,BLACK


# Export Picture Data

In [31]:
df.to_csv(data_raw_folder/'pictures.csv',index=False)

# Download images

In [37]:
df_pictures = pd.read_csv(data_raw_folder/'pictures.csv')
df_pictures.head()


,image,page,color
0,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-ma...,WHITE
1,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/sympho...,WHITE
2,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...,WHITE
3,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...,WHITE
4,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/portra...,WHITE


In [40]:
(df_pictures.image == 'none' ).sum()

67

67 Images have missing image url. Maybe scraping not wait for load this content

In [10]:
df_pictures = df_pictures[(df_pictures.image != 'none' )]

In [11]:
len(df_pictures)

16311

In [12]:
df_pictures['image'] = df_pictures['image'].str.split("\"",expand=True)[1]

In [13]:
df_pictures['index']= df_pictures.index

In [20]:
image_paths =df_pictures['image'].values.tolist()
index_list =df_pictures['index'].values.tolist()

In [34]:
def downloadImg(url,index) -> bool:
    filename = f'{index}.jpg'
    fullpath = data_processed_folder/'img'/filename
    try:
        getImage(url,fullpath)
        return True
    except:
        print(f'Failed to download {url}')
        return False

def parallelDownloadImg(image_paths,index_list) -> None:
    with ProcessPoolExecutor() as executor:
        executor.map(downloadImg, image_paths,index_list)


In [ ]:
parallelDownloadImg(image_paths,index_list)